In [13]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from safetensors.torch import save_file
import os
from tqdm import tqdm

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [ ]:
SEQ_LEN = 256
BATCH_SIZE = 16
EPOCHS = 10
LR = 2e-5
SAVE_EVERY = 2

In [16]:
data = pd.read_csv("H:/SentimentAnalystComment/BERT-base/NTC_SV/NTC_SV_train.csv").dropna()
texts = data["review"].tolist()
labels = LabelEncoder().fit_transform(data["label"])

In [17]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

In [18]:
class FoodyDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(self.labels[idx], dtype=torch.long)
        }

In [19]:
X_train, X_val, y_train, y_val = train_test_split(texts, labels, test_size=0.15, random_state=42)
train_dataset = FoodyDataset(X_train, y_train, tokenizer, SEQ_LEN)
val_dataset = FoodyDataset(X_val, y_val, tokenizer, SEQ_LEN)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

In [20]:
test = pd.read_csv("H:/SentimentAnalystComment/BERT-base/NTC_SV/NTC_SV_test.csv").dropna()
data_test = test["review"].tolist()
label_test = LabelEncoder().fit(test["label"]).transform(test["label"])
test_dataset = FoodyDataset(data_test, label_test, tokenizer, SEQ_LEN)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [21]:
class BERTClassifier(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-multilingual-cased")
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        return self.fc(x)

In [22]:
model = BERTClassifier(num_classes=len(set(labels))).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

In [23]:
os.makedirs("checkpoints", exist_ok=True)

In [24]:
def train_epoch(model, data_loader):
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []
    for batch in tqdm(data_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    return total_loss / len(data_loader), acc

In [ ]:
for epoch in range(EPOCHS):
    loss, acc = train_epoch(model, train_loader)
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {loss:.4f} - Accuracy: {acc:.4f}")

    if (epoch + 1) % SAVE_EVERY == 0:
        save_path = f"checkpoints/bert_base_epoch{epoch+1}.safetensors"
        save_file(model.state_dict(), save_path)
        print(f"✔️ Saved checkpoint at {save_path}")

In [ ]:
model.eval()
preds, trues = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask)
        pred = torch.argmax(outputs, dim=1)
        preds.extend(pred.cpu().numpy())
        trues.extend(labels.cpu().numpy())

print("=== Test Result ===")
print(classification_report(trues, preds))


In [ ]:
save_file(model.state_dict(), "bert_base_foody.safetensors")